In [1]:
import sys
import os
import json

sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "..")))

In [2]:
import json
from adapters.SaucerSwap.adapter.SaucerSwapAdapter import SaucerSwapAdapter

# 1) Configuración
CONFIG_PATH = "/root/Repositorios/idl/adapters/SaucerSwap/config/hedera.saucerswap.yaml"
adapter = SaucerSwapAdapter(CONFIG_PATH)

# 2) Parámetros del swap
# - token_in/token_out: "HBAR" o id HTS "0.0.x"
# - amount: en unidades mínimas del token_in (HBAR/WHBAR 8 decimales, USDC 6 decimales, etc.)
# - fee_bps: tier de la pool (por ejemplo, 1500 = 0.15%)
token_in = "HBAR"
token_out = "0.0.456858"  # USDC en Hedera
amount = 100_000_000      # 1 HBAR si WHBAR tiene 8 decimales
fee_bps = 1500
kind = "exact_in"         # "exact_in" o "exact_out"

# 3) Quote
quote = adapter.get_quote(
    token_in=token_in,
    token_out=token_out,
    amount=amount,
    kind=kind,
    fee_bps=fee_bps,
)
print("Quote:")
print(json.dumps(quote, indent=2))

# 4) Preparación (usa el quote, NO envía la transacción)
prep = adapter.swap_prepare(
    quote,
    slippage_bps=50,  # 0.50%
    deadline_s=300,   # 5 minutos
)
print("\nSwap (prepare):")
print(json.dumps({
    "to": prep["to"],
    "value": prep["value"],
    "gasEstimate": prep["gasEstimate"],
    "data_prefix": prep["data"][:66] + "...",
    "notes": prep.get("notes"),
    "association": prep.get("association"),
    "associated": prep.get("associated"),
    "approve_present": prep.get("approve") is not None
}, indent=2))

# 5) Envío (ejecuta approve si fue preparado y el swap)
result = adapter.swap_send(prep, wait=True)
print("\nResultado envío:")
print(json.dumps(result, indent=2))

# 6) Verificación de éxito
def swap_status_ok(res: dict) -> bool:
    try:
        swap_step = next(s["swap"] for s in res.get("steps", []) if "swap" in s)
    except StopIteration:
        return False
    receipt = swap_step.get("receipt")
    status = receipt and receipt.get("status")
    # Hedera JSON-RPC: éxito suele ser "0x1"
    return str(status) in ("0x1", "1", 1)

ok = swap_status_ok(result)
print("\nSwap OK?:", ok)
if not ok:
    # Si falló, intenta mostrar motivo si existe
    try:
        swap_step = next(s["swap"] for s in result.get("steps", []) if "swap" in s)
        receipt = swap_step.get("receipt", {})
        print("revertReason:", receipt.get("revertReason"))
        print("revertDecoded:", receipt.get("revertDecoded"))
    except Exception:
        pass

Quote:
{
  "amountOut": 236831,
  "sqrtPriceX96AfterList": [
    1626798781467016070907233970729
  ],
  "initializedTicksCrossedList": [
    1
  ],
  "gasEstimate": 91075,
  "path": "0x0000000000000000000000000000000000163b5a0005dc000000000000000000000000000000000006f89a",
  "kind": "exact_in",
  "tokenInRaw": "HBAR",
  "tokenOutRaw": "0.0.456858",
  "requestedAmount": 100000000,
  "feeBps": 1500,
  "routeHops": []
}

Swap (prepare):
{
  "to": "0x00000000000000000000000000000000003c437a",
  "value": 1000000000000000000,
  "gasEstimate": 236977,
  "data_prefix": "0xac9650d800000000000000000000000000000000000000000000000000000000...",
  "notes": [],
  "association": null,
  "associated": true,
  "approve_present": false
}

Resultado envío:
{
  "steps": [
    {
      "swap": {
        "txHash": "0x3ce56889411d23b7fa0374f0500939c62df627989c9aa7221be0d0dec115e829",
        "receipt": {
          "blockHash": "0x89dd3a2e029d45aa1bf007dfac82d4710030c12c00a058f8e7e46160c04671e4",
          "bl